In [5]:
import cassandra
from cassandra.cluster import Cluster

try:
    cluster = Cluster(['127.0.0.1'])

# To establish connection and begin executing queries, need a session
    session = cluster.connect()
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

 Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
 sessionId = 338, and itemInSession = 4
 
 The table for this query uses the session_id as the partitioning key, to allocate same session_id to thesame node and the item_insession is use for sorting the record in each node. With this, reading out the artist, song_title, and song_length heard during a session only requires identifying the node the value "338" is partition and looking up the sorted column (item_insession) for the corresponding value "4". 

In [6]:
try:
    select_query_1 ="SELECT artist,song_title,song_length FROM artist_library \
    WHERE session_id= 338 AND item_insession= 4" 
 
    results = session.execute(select_query_1)
    for row in results:
        print(row.artist,'|',row.song_title,'|',row.song_length)
except Exception as e:
    print(e)

Faithless | Music Matters (Mark Knight Dub) | 495.3073


Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

The table created for this query, uses the user_id and session_id as the composite partition key to partition the records. A better and more balance distribution is obtain with this composite key, than using a single column like the user_id, since some users visits the app more frequent than others. Adding the session_id to the partition key will force thesame user_id with same session_id to be on thesame node thereby reducing the initial skewness (using only the user_id). Obtaining the query result requires hashing the partition key values "10" and "182" and identifying the node it is partition. The item_insession column is also used to sort the records in each node, making it efficient for selection. 

In [7]:
try:
    select_query_2 ="SELECT artist,song_title,user_first_name,user_last_name FROM user_library \
    WHERE user_id= 10 AND session_id= 182" 
 
    results = session.execute(select_query_2)
    for row in results:
        print(row.artist,'|',row.song_title,'|',row.user_first_name,'|',row.user_last_name)
except Exception as e:
    print(e)

Down To The Bone | Keep On Keepin' On | Sylvie | Cruz
Three Drives | Greece 2000 | Sylvie | Cruz
Sebastien Tellier | Kilometer | Sylvie | Cruz
Lonnie Gordon | Catch You Baby (Steve Pitron & Max Sanna Radio Edit) | Sylvie | Cruz


 Query 3: Give me every user name (first and last) in my music app history who listened to the song 
 'All Hands Against His Own'
 
 The table created for this query uses the song_title and user_id as the primary key, which is assumed to be unique. This will not be unique if a user can repeat a song, another suggestion will be to add the session_id to the primary key, meaning a user can't repeat thesame song in thesame session. For a case where a user can't repeat thesame song, the partitioning key being the song_title is used to partition the records, and obtaining the result of users who listened to a particular song will only invole records from a single node.

In [8]:
try:
    select_query_3 ="SELECT user_first_name,user_last_name FROM song_library \
    WHERE song_title = 'All Hands Against His Own' " 
 
    results = session.execute(select_query_3)
    for row in results:
        print(row.user_first_name,'|',row.user_last_name)
except Exception as e:
    print(e)

Jacqueline | Lynch
Tegan | Levine
Sara | Johnson


In [9]:
session.shutdown()
cluster.shutdown()